# Singstat and Data.gov.sg Information crawler.
This notebook is used for pulling and consolidating datasets found in both sites

## Install dependencies for pulling data sources information

### Notes: BeautifulSoup can only handle static website content scraping. Selenium library is required together with beautifulsoup to read all dynamically loaded content which is the case for singstat and data.gov.sg.

In [1]:
#!pip install requests selenium

Library imports

In [18]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException, ElementClickInterceptedException
from datetime import datetime
from typing import NewType
import time
import pandas as pd
# Define options for webdriver
chrome_options = Options()

Try to click Data.gov.sg and pull data information
Attempt to load as much information as possible before one shot scraping. TO avoid issues, please do not minimize the page when executing

In [45]:
# Helper function
def load_all_data(dataset_on_display: list,
                  total_results: int,
                  load_more_button: object,
                  datasets_div_xpath:str,
                  load_more_xpath:str,
                  driver: NewType):
    while len(dataset_on_display) < total_results and load_more_button:
        print("Loading more data")
        # Count displayed dataset

        dataset_on_display = driver.find_elements(by=By.XPATH, value=datasets_div_xpath)
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, 
            load_more_xpath)))
        except TimeoutException:
            print("No load more button available after waiting. Assuming the end of page")


        try:
            load_more_button = driver.find_element(by=By.XPATH, value=load_more_xpath)
            print(f"Current display data: {len(dataset_on_display)}/{total_results}")
            load_more_button.click()
        except NoSuchElementException:
            print("Unable to click load more button due to no such element")
        except ElementClickInterceptedException:
            print("Encountered interference with clicking...")

In [46]:
URL = "https://beta.data.gov.sg/datasets?sort=Last%20updated"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)
driver.get(URL)

# XPATH
load_more_xpath = "//button[contains(@class, 'chakra-button') and text()='Load more']"
results_xpath = "//div[@class='css-no1clx']/p[contains(@class, 'chakra-text')][2]"
datasets_div_xpath = "//a[@class='chakra-link css-mfiv8d']"

# Webpage wait
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, load_more_xpath)))

# FInd elements
total_results = driver.find_element(by=By.XPATH, value=results_xpath)
if total_results:
    # Results value in the string format of xxx,xxx. Hence need to remove commas
    total_results_text = total_results.text
    total_results = total_results_text.replace(",","")
    total_results = int(total_results)
else:
    print("Cant extract any results information from page")
    exit()
load_more_button = driver.find_element(by=By.XPATH, value=load_more_xpath)
# Count displayed dataset
dataset_on_display = driver.find_elements(by=By.XPATH, value=datasets_div_xpath)

# CHeck current dataset display numbers
if load_more_button and total_results:
    print(f"Found load more button with {total_results} total results available")

# This function will force expansion of web page by clicking on load more button until there isnt any.
   
load_all_data(dataset_on_display=dataset_on_display,
              total_results=total_results,
              load_more_button=load_more_button,
              datasets_div_xpath=datasets_div_xpath,
              load_more_xpath=load_more_xpath,
              driver=driver)

Found load more button with 4832 total results available
Loading more data
Current display data: 20/4832
Loading more data
Current display data: 20/4832
Loading more data
Current display data: 40/4832
Loading more data
Current display data: 60/4832
Loading more data
Current display data: 80/4832
Loading more data
Current display data: 100/4832
Loading more data
Current display data: 120/4832
Loading more data
Current display data: 140/4832
Loading more data
Current display data: 160/4832
Loading more data
Current display data: 180/4832
Loading more data
Current display data: 200/4832
Loading more data
Current display data: 220/4832
Loading more data
Current display data: 240/4832
Loading more data
Current display data: 260/4832
Loading more data
Current display data: 280/4832
Loading more data
Current display data: 300/4832
Loading more data
Current display data: 320/4832
Loading more data
Current display data: 340/4832
Loading more data
Current display data: 360/4832
Loading more data

NameError: name 'ElementClickInterceptedException' is not defined

## Do an overall count after load more has been exhausted

In [6]:
total_dataset_on_display = driver.find_elements(by=By.XPATH, value=datasets_div_xpath)
print(len(total_dataset_on_display))

20


## Extract metadata of dataset for storing purpose

In [7]:
# Expand the list
#Collection
collection_tracker_dict = {}

# Dataset 
dataset_tracker_dict= {}

get_dataset_name_xpath = ".//div/div/p"
get_metadata_info_xpath = ".//div/div/div[contains(@class, 'chakra-wrap')]/ul/p[contains(@class, 'chakra-text')]"
for dataset in total_dataset_on_display:
    try:
        href = dataset.get_attribute('href')
        print("Dataset link:")
        print(href)
    except AttributeError:
        print("No href info found.")
    
    try:
        name = dataset.find_element(by=By.XPATH, value=get_dataset_name_xpath)
        name = name.text
        print("Dataset name:")
        print(name) 
    except NoSuchElementException:
        print("No name information found")

    # Multiple metadata information
    metadata_list = dataset.find_elements(by=By.XPATH, value=get_metadata_info_xpath)
    # Filter alternate elements
    metadata_list = [metadata.text for i,metadata in enumerate(metadata_list) if i%2==0]
    print(metadata_list)

    # In the case where metadata list only has 3 elements (no date), include None as info to ensure ease of outputing all dataset info into csv file.
    if len(metadata_list) == 3:
        metadata_list.insert(0, "No Date info")
    #Insert href info
    metadata_list.append(href)

    # MEtadata construct influence whether we are dealing with collections or dataset
    if any(" dataset" in metadata for metadata in metadata_list):
        collection_tracker_dict[name] = metadata_list
    else:
        dataset_tracker_dict[name] = metadata_list
    print()   

driver.quit()

Dataset link:
https://beta.data.gov.sg/datasets/d_c31c965da0c7c4bdc897eebb6747486a/view
Dataset name:
Notices under other Acts 2024
['Updated 13 minutes ago', 'CSV', 'Ministry of Communications and Information (MCI)']

Dataset link:
https://beta.data.gov.sg/datasets/d_2c7c87c899bf1264f34eba18890518e5/view
Dataset name:
Notices under the Constitution 2024
['Updated 13 minutes ago', 'CSV', 'Ministry of Communications and Information (MCI)']

Dataset link:
https://beta.data.gov.sg/datasets/d_edb599302d97748dc95ef4fe16fcecd5/view
Dataset name:
Death 2024
['Updated 13 minutes ago', 'CSV', 'Ministry of Communications and Information (MCI)']

Dataset link:
https://beta.data.gov.sg/datasets/d_830ca47f48adfac75b79dc7d0efd8744/view
Dataset name:
Leave 2024
['Updated 13 minutes ago', 'CSV', 'Ministry of Communications and Information (MCI)']

Dataset link:
https://beta.data.gov.sg/datasets/d_8f848100c27d6d4c534769130dbba7ff/view
Dataset name:
Supplements (Current Notices)
['Updated 13 minutes ago

In [9]:
print(len(dataset_tracker_dict), len(collection_tracker_dict))

20 0


In [20]:
## Convert dataset dictionary to dataframe
if dataset_tracker_dict:
    df = pd.DataFrame.from_dict(dataset_tracker_dict, orient="index", columns=["Data period", "Last updated", "Datatype", "Source", "Dataset url"])

    datetime_now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    file_name_date = datetime_now().strfttme("%d%m%Y_%H%M%S")

    df["Date_of_check"] = datetime_now
    df.index.name = "Datasets"
    df.head()

    dataset_filename = f"Datagovsg_dataset_{file_name_date}.csv"
    df.to_csv(dataset_filename,index=True)


## Convert collections dictioanry to dataframe
if collection_tracker_dict:
    df = pd.DataFrame.from_dict(dataset_tracker_dict, orient="index", columns=["Data period", "Number of datasets", "Datatype", "Source", "Collections url"])

    datetime_now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    file_name_date = datetime_now().strfttme("%d%m%Y_%H%M%S")

    df["Date_of_check"] = datetime_now
    df.index.name = "Collections"
    df.head()

    dataset_filename = f"Datagovsg_collections_{file_name_date}.csv"
    df.to_csv(dataset_filename,index=True)


## Handling SingStat tablebuilder as source

In [36]:
new_url="https://tablebuilder.singstat.gov.sg/"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)
#maximize browser
driver.get(new_url)

# Wait popup
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Maybe Later']"))).click()

# Wait removal of popup
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Expand all']"))).click()

singstat_data_dict = {}

# Navigate through the body after expand all

# We have 6 headers
main_categories_xpath = "//span[contains(@class, 'irmCui')]"
main_categories = driver.find_elements(by=By.XPATH, value=main_categories_xpath)

print(f"Total sections: {len(main_categories)}")
for i, option in enumerate(main_categories):
    
    main_category = option.text
    print(f"Main category: {main_category}")

    sub_category_caret_xpath = f"//*[contains(text(), '{option.text}')]/../../../following-sibling::div/div"

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, sub_category_caret_xpath)))
    
    # Get the caret-right
    sub_categories_carets = option.find_elements(by=By.XPATH, value=sub_category_caret_xpath)

    print("Total sub categories:")
    print(len(sub_categories_carets))

    # Nested loop to extract subcategories
    for sub_categories_caret in sub_categories_carets:
        # Expand each subcategories
        sub_categories_caret.click()

        # Get the text of sub category
        text_div= ".//a/div/div[2]/div"
        sub_category_text_name = sub_categories_caret.find_element(by=By.XPATH, value=text_div).text

        # THere are nested sub categories
        sub_sub_category_caret_xpath =".//div[contains(@class, 'sgds-accordion-body is-open')]/div"

        # FInd number of such
        sub_sub_categories_list = sub_categories_caret.find_elements(by=By.XPATH, value=sub_sub_category_caret_xpath)
        print(f"Sub category: {sub_category_text_name} has {len(sub_sub_categories_list)} sub-sub categories")

        # Inner nested loop for sub-sub categories
        for sub_sub_cat in sub_sub_categories_list:
            sub_sub_cat.click()

            sub_sub_category_text_name = sub_sub_cat.find_element(by=By.XPATH, value = text_div).text

            sub_sub_category_file_xpath = ".//div/a"
            file_elements_list = sub_sub_cat.find_elements(by=By.XPATH, value=sub_sub_category_file_xpath)
            
            print(f"Sub-sub category: {sub_sub_category_text_name} has {len(file_elements_list)} dataset")
            # Processing or the anchor tags <a> to obtain href info and text

            if file_elements_list:
                for file in file_elements_list:
                    href = file.get_attribute('href')
                    text = file.text

                    dataset_hierarachy_str = main_category + ">" + sub_category_text_name + ">" + sub_sub_category_text_name + ">" + text

                    singstat_data_dict[dataset_hierarachy_str] = [href]
        print()
# Quit browser to free resources
driver.quit()

Total sections: 6
Main category: Economy & Prices
Total sub categories:
10
Sub category: Balance of Payments (BOP) has 2 sub-sub categories
Sub-sub category: Singapore's Balance of Payments (BOP) has 3 dataset
Sub-sub category: Singapore's Inward Direct Investment Flows has 5 dataset

Sub category: External Debt has 1 sub-sub categories
Sub-sub category: Singapore's External Debt has 1 dataset

Sub category: Household Sector Balance Sheet has 1 sub-sub categories
Sub-sub category: Household Net Worth, Assets and Liabilities has 1 dataset

Sub category: International Investment Position (IIP) has 3 sub-sub categories
Sub-sub category: Singapore's International Investment Position (IIP) has 1 dataset
Sub-sub category: Singapore's Portfolio Investment Assets has 1 dataset
Sub-sub category: Singapore’s Portfolio Investment Liabilities has 1 dataset

Sub category: Labour, Employment, Wages and Productivity has 12 sub-sub categories
Sub-sub category: CPF Members and Employers has 4 dataset
S

In [40]:
# Check datasets amount
print(len(singstat_data_dict))

df = pd.DataFrame.from_dict(singstat_data_dict, orient="index", columns=["Dataset url"])
df.reset_index(inplace=True)

dataset_category_list = ["Theme", "Categories", "SubCategories", "Dataset"]
df[dataset_category_list] = df["index"].str.split(">",expand=True)

datetime_now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
file_name_date = datetime_now().strfttme("%d%m%Y_%H%M%S")

df["Date_of_check"] = datetime_now

dataset_filename = f"Singstat_dataset_{file_name_date}.csv"
df.to_csv(dataset_filename,index=True)

df.head()

867


,index,Dataset url
0,Economy & Prices>Balance of Payments (BOP)>Sin...,https://tablebuilder.singstat.gov.sg/table/TS/...
1,Economy & Prices>Balance of Payments (BOP)>Sin...,https://tablebuilder.singstat.gov.sg/table/TS/...
2,Economy & Prices>Balance of Payments (BOP)>Sin...,https://tablebuilder.singstat.gov.sg/table/TS/...
3,Economy & Prices>Balance of Payments (BOP)>Sin...,https://tablebuilder.singstat.gov.sg/table/TS/...
4,Economy & Prices>Balance of Payments (BOP)>Sin...,https://tablebuilder.singstat.gov.sg/table/TS/...


## Note on filtering from driver.find_elements (big bracket over):
Example 3rd element of all find elements:
detailsAccordion = driver.find_elements(by=By.XPATH, value="(//*[contains(@class, 'is-open sgds-accordion-body')])[2]")

In [98]:
detailsAccordion = driver.find_elements(by=By.XPATH, value="(//*[contains(@class, 'is-open sgds-accordion-body')])[2]")
detailsAccordion

[<selenium.webdriver.remote.webelement.WebElement (session="081dccc68fc703cfa846c88cc73ba243", element="f.70A807E70882149E9A2ADEECF7E68E8D.d.946879DEBFEF4C7C62F943CB1E5E1C1D.e.191")>]